In [ ]:
!pip install requests beautifulsoup4 tqdm

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time
import random

BASE_URL = "https://www.pakwheels.com/used-cars/search/-/"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def scrape_page(page_num, retries=3, backoff=1):
    """
    Scrapes one page of PakWheels listings with retry & exponential backoff.
    """
    url = f"{BASE_URL}?page={page_num}&_pjax=%5Bdata-pjax-container%5D"

    for attempt in range(retries):
        try:
            response = requests.get(url, headers=HEADERS, timeout=15)

            if response.status_code != 200:
                raise Exception(f"HTTP {response.status_code}")

            soup = BeautifulSoup(response.text, "html.parser")
            listings = soup.find_all("li", class_="classified-listing")
            page_data = []

            for listing in listings:
                try:
                    # Title
                    title_tag = listing.find("a", class_="car-name ad-detail-path")
                    title = title_tag.get("title") if title_tag else None

                    # Price
                    price_tag = listing.find("div", class_="price-details generic-dark-grey")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    # Specs (year, mileage, fuel, engine, transmission)
                    specs_ul = listing.find("ul", class_="list-unstyled search-vehicle-info-2 fs13")
                    specs = specs_ul.find_all("li") if specs_ul else []

                    year = specs[0].get_text(strip=True) if len(specs) > 0 else None
                    mileage = specs[1].get_text(strip=True) if len(specs) > 1 else None
                    fuel = specs[2].get_text(strip=True) if len(specs) > 2 else None
                    engine = specs[3].get_text(strip=True) if len(specs) > 3 else None
                    transmission = specs[4].get_text(strip=True) if len(specs) > 4 else None

                    page_data.append({
                        "Title": title,
                        "Price": price,
                        "Year": year,
                        "Mileage": mileage,
                        "Fuel": fuel,
                        "Engine": engine,
                        "Transmission": transmission
                    })
                except:
                    continue
            return page_data

        except Exception as e:
            wait = backoff * (2 ** attempt) + random.uniform(0, 0.5)  # jitter
            print(f"⚠️ Page {page_num} failed (attempt {attempt+1}/{retries}): {e}, retrying in {wait:.1f}s")
            time.sleep(wait)

    print(f"❌ Skipped page {page_num} after {retries} failed attempts")
    return []


def scrape_all(total_pages=2590, workers=20):
    """
    Scrape all pages in parallel using threads + retries
    """
    all_data = []
    with ThreadPoolExecutor(max_workers=workers) as executor:
        futures = {executor.submit(scrape_page, page): page for page in range(1, total_pages+1)}

        for future in tqdm(as_completed(futures), total=total_pages, desc="Scraping"):
            all_data.extend(future.result())

    return pd.DataFrame(all_data)


# Run in Colab
if __name__ == "__main__":
    df = scrape_all(total_pages=2591, workers=10)  # adjust workers if blocked
    print(df.head())
    df.to_csv("pakwheels_cars.csv", index=False)
    print("✅ Data saved to pakwheels_cars.csv")

Scraping: 100%|██████████| 2591/2591 [13:09<00:00,  3.28it/s]


                                               Title          Price  Year  \
0  Porsche Cayenne  2016 S E-Hybrid Platinum Edition  PKR 2.35crore  2016   
1                       Proton X70  2022 Premium FWD     PKR 70lacs  2022   
2                             FAW X-PV  2014 Dual AC      PKR 7lacs  2014   
3                         Honda N Wgn  2023 G Turbo      PKR 40lacs  2023   
4                           Suzuki Wagon R  2017 VXR   PKR 17.5lacs  2017   

      Mileage    Fuel   Engine Transmission  
0   75,943 km  Hybrid  3000 cc    Automatic  
1   12,319 km  Petrol  1500 cc    Automatic  
2  331,849 km  Petrol  1000 cc       Manual  
3   25,762 km  Petrol   660 cc    Automatic  
4  147,219 km  Petrol  1000 cc       Manual  
✅ Data saved to pakwheels_cars.csv
